In [1]:
import pandas as pd
df = pd.read_csv('0521_price_volatility_ver3.csv')
df

,stock_symbol,title,date,paragraphs,1_day_change_rate
0,TTWO,Take-Two Interactive Software (TTWO) Q4 2024 E...,2024-05-16,p.m. ET\n\nOperator\n\nGreetings and welcome ...,0.012048
1,AMAT,Applied Materials (AMAT) Q2 2024 Earnings Call...,2024-05-16,perator\n\nWelcome to the Applied Materials ea...,-0.009111
2,DXC,DXC Technology (DXC) Q4 2024 Earnings Call Tra...,2024-05-16,"ator\n\nHello, and welcome to the DXC Technolo...",-0.169014
3,LSPD,Lightspeed Commerce (LSPD) Q4 2024 Earnings Ca...,2024-05-16,"Operator\n\nThank you for standing by, and wel...",0.021192
4,JD,JD.com (JD) Q1 2024 Earnings Call Transcript,2024-05-16,", and thank you for standing by for JD.com's f...",0.029180
...,...,...,...,...,...
993,PYCR,Paycor HCM (PYCR) Q1 2024 Earnings Call Transc...,2023-11-08,"r\n\nLadies and gentlemen, thank you for stand...",0.044258
994,DUOL,Duolingo (DUOL) Q3 2023 Earnings Call Transcript,2023-11-08,"van\n\nGood evening, everyone. If you haven't ...",0.213726
995,AFRM,Affirm (AFRM) Q1 2024 Earnings Call Transcript,2023-11-08,"d afternoon, ladies and gentlemen. Thank you f...",0.142463
996,CART,Instacart (CART) Q3 2023 Earnings Call Transcript,2023-11-08,Good day and thank you for standing by. Welcom...,-0.101322


In [4]:
df['paragraphs'][0]

" p.m. ET\n\nOperator\n\nGreetings and welcome to the Take-Two fourth quarter and fiscal year 2024 earnings call. At this time, all participants are in a listen-only mode. A brief question-and-answer session will follow the formal presentation. [Operator instructions] As a reminder, this conference is being recorded.\n\nIt is now my pleasure to introduce your host, Nicole Shevins, senior vice president of investor relations and corporate communications. Thank you. You may begin.\n\nNicole Shevins--Senior Vice President of Investor Relations and Corporate Communications\n\nGood afternoon. Thank you for joining our conference call to discuss our results for the fourth quarter and fiscal year 2024 ended March 31, 2024. Today's call will be led by Strauss Zelnick, Take-Two's chairman and chief executive officer; Karl Slatoff, our president; and Lainie Goldstein, our chief financial officer. We will be available to answer your questions during the Q&A session following our prepared remarks.

In [8]:
%%time
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 讀取數據
train_df = df.copy()

# 文本預處理函數
def preprocess_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    return text

train_df['paragraphs'] = train_df['paragraphs'].apply(preprocess_text)

# 特徵提取 - 使用TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(train_df['paragraphs'])

# 標籤
y = train_df['1_day_change_rate']

# 拆分訓練和測試數據
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練模型
model = LinearRegression()
model.fit(X_train, y_train)

# 預測
y_pred = model.predict(X_test)

# 評估模型
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# 查看部分預測結果
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results.head())

ValueError: Input y contains NaN.